In [ ]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import sqlite3

In [2]:
url = "https://docs.google.com/spreadsheets/d/1zLNAuRqPauss00HDz4XbTH2HqsCzMe0pR8QmD1K8jk8/pubhtml/sheet?headers=false&chrome=false&gid=0&range=A3:I437"

In [3]:
df = pd.read_html(url)

In [4]:
df = df[0]

In [5]:
df.rename(inplace=True, columns={1:'District', 5:'2016', 7:'2012', 9:'2008'})

In [6]:
df = df[["District", "2016", "2012", "2008"]]
df.head()

,District,2016,2012,2008
0,AK-AL,52.8,55.3,59.7
1,AL-01,63.5,61.8,60.9
2,AL-02,64.9,62.9,64.5
3,AL-03,65.3,62.3,62.6
4,AL-04,80.4,74.8,73.3


In [7]:
geoDf = pd.read_json('GeoData.js')
geoDf = geoDf.transpose()

In [8]:
geoDf.head()

,0,1,2,3
MO08,-92.909347,35.995683,-89.098968,38.377400
IA03,-96.019543,40.570706,-93.327886,41.863728
IL13,-90.936728,38.661213,-88.123219,40.539467
IL16,-89.868384,40.398478,-87.525661,42.500563
IL06,-88.403606,41.743478,-87.907793,42.241746


In [10]:
def fixDistricts(district):
    state = district[0:2]
    distNum = district[3:5]
    if (distNum == 'AL'):
        distNum = '00'
    newDist = state + distNum
    return newDist

In [11]:
df['District'] = df['District'].apply(lambda x: fixDistricts(x))

In [12]:
def addGeo(district):
    geoList = []
    for geo in geoDf.loc[district]:
        geoList.append(geo)
    return str(geoList)

In [13]:
df['GeoData'] = df['District'].apply(lambda x: addGeo(x))

In [14]:
df = df[['District', '2016', '2012', '2008', 'GeoData']]
df.head()

,District,2016,2012,2008,GeoData
0,AK00,52.8,55.3,59.7,"[-179.231086, 51.175092, 179.859681, 71.441059]"
1,AL01,63.5,61.8,60.9,"[-88.46443, 30.144425, -86.688063, 31.831795]"
2,AL02,64.9,62.9,64.5,"[-87.427516, 30.992859, -84.999428, 32.769225]"
3,AL03,65.3,62.3,62.6,"[-86.577799, 32.061708, -84.888246, 34.39592]"
4,AL04,80.4,74.8,73.3,"[-88.274619, 33.128724, -85.474024, 34.906694]"


In [15]:
connection = sqlite3.Connection('db_file.db')

In [19]:
df.to_sql(name='DistrictData', con=connection, index=True, if_exists='replace')